In [1]:
!pip -q install -U condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 65.2 MB/s eta 0:00:00


In [2]:
!conda install -y ipykernel

Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ipykernel


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    asttokens-3.0.1            |     pyhd8ed1ab_0          28 KB  conda-forge
    comm-0.2.3                 |     pyhe01879c_0          14 KB  conda-forge
    debugpy-1.8.18             |  py311hc665b79_0         2.6 MB  conda-forge
    decorator-5.2.1            |     pyhd8ed1ab_0          14 KB  conda-forge
    executing-2.2.1            |     pyhd8ed1ab_0          30 KB  conda-forge
    ipykernel-7.1.0            |     pyha191276_0         131 KB  conda-forge
    ip

In [3]:
!python -m ipykernel install --user --name=conda-py311 --display-name "Python (conda-py311)"

0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Installed kernelspec conda-py311 in /root/.local/share/jupyter/kernels/conda-py311


In [3]:
import sys
print(sys.executable, sys.version)
from rdkit import Chem
print("RDKit OK", Chem.MolFromSmiles("CCO") is not None)

/usr/bin/python3 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
RDKit OK True


In [5]:
!pip install torch
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.2 MB/s eta 0:00:00


In [14]:
!pip install duckdb
!pip install pandas

In [15]:
!pip install -q kaggle

In [52]:
import os
for k in ["KAGGLE_USERNAME", "KAGGLE_KEY", "KAGGLE_CONFIG_DIR", "HTTP_PROXY", "HTTPS_PROXY"]:
    if k in os.environ:
        print("removing env", k)
        os.environ.pop(k, None)

os.environ["KAGGLE_CONFIG_DIR"] = "/root/.kaggle"
print("KAGGLE_CONFIG_DIR=", os.environ["KAGGLE_CONFIG_DIR"])


removing env KAGGLE_USERNAME
removing env KAGGLE_KEY
removing env KAGGLE_CONFIG_DIR
KAGGLE_CONFIG_DIR= /root/.kaggle


In [49]:

import os, json
p="/root/.kaggle/kaggle.json"
print("exists", os.path.exists(p))
print("KAGGLE_CONFIG_DIR", os.environ.get("KAGGLE_CONFIG_DIR"))
print("HOME", os.environ.get("HOME"))
print("file", p)
print("username", json.load(open(p))["username"])

exists True
KAGGLE_CONFIG_DIR /root/.kaggle
HOME /root
file /root/.kaggle/kaggle.json
username sofiasampara


In [58]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [61]:
!ls -lh /content/drive/MyDrive/leash-BELKA.zip


-rw------- 1 root root 4.2G Dec 14 17:04 /content/drive/MyDrive/leash-BELKA.zip


In [63]:
!mkdir -p /content/data
!unzip -q "/content/drive/MyDrive/leash-BELKA.zip" -d /content/data


In [64]:
!ls /content/data | head


sample_submission.csv
test.csv
test.parquet
train.csv
train.parquet


In [67]:
train_path = "data/train.parquet"
test_path  = "data/test.parquet"

print("Using train_path:", train_path)
print("Using test_path :", test_path)


Using train_path: data/train.parquet
Using test_path : data/test.parquet


In [6]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from rdkit import Chem
from rdkit.Chem.Scaffolds import MurckoScaffold

from sklearn.model_selection import GroupKFold
from sklearn.metrics import average_precision_score

from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GINConv, global_add_pool


In [11]:
from collections import defaultdict

In [68]:
import duckdb
import pandas as pd

N_NEG = 30000
N_POS = 30000
SEED = 42

con = duckdb.connect()

df_train = con.query(f"""
    (SELECT id, molecule_smiles, protein_name, binds
     FROM parquet_scan('{train_path}')
     WHERE binds = 0
     ORDER BY random()
     LIMIT {N_NEG})
    UNION ALL
    (SELECT id, molecule_smiles, protein_name, binds
     FROM parquet_scan('{train_path}')
     WHERE binds = 1
     ORDER BY random()
     LIMIT {N_POS})
""").df()

print("df_train:", df_train.shape)
print(df_train["binds"].value_counts())

con.close()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

df_train: (60000, 4)
binds
0    30000
1    30000
Name: count, dtype: int64


In [69]:
import numpy as np
import torch
from rdkit import Chem
from torch_geometric.data import Data

proteins = sorted(df_train["protein_name"].astype(str).unique().tolist())
protein_to_idx = {p:i for i,p in enumerate(proteins)}
n_proteins = len(protein_to_idx)
print("n_proteins:", n_proteins)

BOND_TYPES = {
    Chem.BondType.SINGLE: 0,
    Chem.BondType.DOUBLE: 1,
    Chem.BondType.TRIPLE: 2,
    Chem.BondType.AROMATIC: 3
}

def atom_features(atom) -> np.ndarray:
    return np.array([
        atom.GetAtomicNum(),
        atom.GetTotalDegree(),
        int(atom.GetIsAromatic()),      # aromatic
        atom.GetFormalCharge(),         # charge
        int(atom.GetHybridization()),   # hybridization enum -> int
    ], dtype=np.float32)

def bond_features(bond) -> np.ndarray:
    bt = BOND_TYPES.get(bond.GetBondType(), 0)
    return np.array([
        bt,
        int(bond.GetIsConjugated()),
        int(bond.IsInRing()),
    ], dtype=np.float32)

def smiles_to_pyg(smiles: str):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    x = np.stack([atom_features(a) for a in mol.GetAtoms()], axis=0)
    x = torch.tensor(x, dtype=torch.float)

    edge_index = []
    edge_attr = []
    for b in mol.GetBonds():
        i, j = b.GetBeginAtomIdx(), b.GetEndAtomIdx()
        bf = bond_features(b)
        edge_index.append([i, j]); edge_attr.append(bf)
        edge_index.append([j, i]); edge_attr.append(bf)

    if len(edge_index) == 0:
        edge_index = torch.zeros((2, 0), dtype=torch.long)
        edge_attr  = torch.zeros((0, 3), dtype=torch.float)
    else:
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
        edge_attr  = torch.tensor(np.stack(edge_attr, axis=0), dtype=torch.float)

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr)


n_proteins: 3


In [70]:
from tqdm import tqdm

train_graphs = []
bad = 0

for row in tqdm(df_train.itertuples(index=False), total=len(df_train)):
    g = smiles_to_pyg(row.molecule_smiles)
    if g is None:
        bad += 1
        continue

    g.y = torch.tensor(float(row.binds), dtype=torch.float)        # label
    g.prot = torch.tensor(protein_to_idx[str(row.protein_name)], dtype=torch.long)
    g.id = int(row.id)
    train_graphs.append(g)

print("built train_graphs:", len(train_graphs), "bad_smiles:", bad)

torch.save(train_graphs, "train_graphs.pt")
print("Saved train_graphs.pt")


100%|██████████| 60000/60000 [01:49<00:00, 550.09it/s]


built train_graphs: 60000 bad_smiles: 0
Saved train_graphs.pt


In [71]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GINEConv, global_mean_pool

class GNNProtein(nn.Module):
    def __init__(self, node_in=5, edge_in=3, hidden=128, n_layers=3, n_proteins=3, prot_dim=32, dropout=0.2):
        super().__init__()
        self.node_lin = nn.Linear(node_in, hidden)
        self.edge_lin = nn.Linear(edge_in, hidden)

        self.convs = nn.ModuleList()
        for _ in range(n_layers):
            mlp = nn.Sequential(
                nn.Linear(hidden, hidden),
                nn.ReLU(),
                nn.Linear(hidden, hidden),
            )
            self.convs.append(GINEConv(nn=mlp, edge_dim=hidden))

        self.prot_emb = nn.Embedding(n_proteins, prot_dim)

        self.head = nn.Sequential(
            nn.Linear(hidden + prot_dim, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1)
        )

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        batch = data.batch

        x = self.node_lin(x)
        e = self.edge_lin(edge_attr)

        for conv in self.convs:
            x = conv(x, edge_index, e)
            x = F.relu(x)

        mol = global_mean_pool(x, batch)
        prot = self.prot_emb(data.prot)

        h = torch.cat([mol, prot], dim=1)
        logits = self.head(h).squeeze(1)
        return logits


In [73]:
import numpy as np
import torch
from torch_geometric.loader import DataLoader
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import average_precision_score

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

y = np.array([int(g.y.item()) for g in train_graphs], dtype=int)

N_SPLITS = 5
EPOCHS = 20
BS_TRAIN = 64
BS_VAL = 256
LR = 1e-3

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

oof_pred = np.zeros(len(train_graphs), dtype=np.float32)
fold_models = []

def predict_proba(model, loader):
    model.eval()
    out = []
    with torch.no_grad():
        for batch in loader:
            batch = batch.to(device)
            logits = model(batch)
            p = torch.sigmoid(logits).detach().cpu().numpy()
            out.append(p)
    return np.concatenate(out, axis=0)

for fold, (tr_idx, va_idx) in enumerate(skf.split(np.zeros(len(y)), y), 1):
    tr_ds = [train_graphs[i] for i in tr_idx]
    va_ds = [train_graphs[i] for i in va_idx]

    tr_loader = DataLoader(tr_ds, batch_size=BS_TRAIN, shuffle=True)
    va_loader = DataLoader(va_ds, batch_size=BS_VAL, shuffle=False)

    model = GNNProtein(
        node_in=5, edge_in=3, hidden=128, n_layers=3,
        n_proteins=n_proteins, prot_dim=32, dropout=0.2
    ).to(device)

    opt = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)
    loss_fn = torch.nn.BCEWithLogitsLoss()

    best_ap = -1
    best_state = None

    for ep in range(1, EPOCHS+1):
        model.train()
        total_loss = 0.0

        for batch in tr_loader:
            batch = batch.to(device)
            opt.zero_grad()
            logits = model(batch)
            loss = loss_fn(logits, batch.y)
            loss.backward()
            opt.step()
            total_loss += float(loss.item()) * batch.num_graphs

        p_va = predict_proba(model, va_loader)
        y_va = y[va_idx]
        ap = average_precision_score(y_va, p_va)

        print(f"Fold {fold} | Epoch {ep}/{EPOCHS} | loss={total_loss/len(tr_ds):.4f} | val_AP={ap:.5f}")

        if ap > best_ap:
            best_ap = ap
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

    model.load_state_dict(best_state)
    fold_models.append(model)

    p_va = predict_proba(model, va_loader)
    oof_pred[va_idx] = p_va.astype(np.float32)

    print(f"Fold {fold} best AP: {best_ap:.5f}\n")

oof_ap = average_precision_score(y, oof_pred)
print("OOF AP:", oof_ap)

np.save("oof_gnn.npy", oof_pred)
print("Saved oof_gnn.npy")


device: cuda
Fold 1 | Epoch 1/20 | loss=0.6341 | val_AP=0.77116
Fold 1 | Epoch 2/20 | loss=0.5677 | val_AP=0.79536
Fold 1 | Epoch 3/20 | loss=0.5395 | val_AP=0.82960
Fold 1 | Epoch 4/20 | loss=0.5064 | val_AP=0.84843
Fold 1 | Epoch 5/20 | loss=0.4788 | val_AP=0.87262
Fold 1 | Epoch 6/20 | loss=0.4549 | val_AP=0.88488
Fold 1 | Epoch 7/20 | loss=0.4368 | val_AP=0.89451
Fold 1 | Epoch 8/20 | loss=0.4277 | val_AP=0.89913
Fold 1 | Epoch 9/20 | loss=0.4109 | val_AP=0.90718
Fold 1 | Epoch 10/20 | loss=0.4003 | val_AP=0.91174
Fold 1 | Epoch 11/20 | loss=0.3859 | val_AP=0.91852
Fold 1 | Epoch 12/20 | loss=0.3751 | val_AP=0.92210
Fold 1 | Epoch 13/20 | loss=0.3579 | val_AP=0.92736
Fold 1 | Epoch 14/20 | loss=0.3438 | val_AP=0.93060
Fold 1 | Epoch 15/20 | loss=0.3303 | val_AP=0.92787
Fold 1 | Epoch 16/20 | loss=0.3178 | val_AP=0.93976
Fold 1 | Epoch 17/20 | loss=0.3059 | val_AP=0.94493
Fold 1 | Epoch 18/20 | loss=0.2957 | val_AP=0.94837
Fold 1 | Epoch 19/20 | loss=0.2908 | val_AP=0.94914
Fold 1 |

In [74]:
protein_to_idx2 = {"<UNK>": 0}
for p, i in protein_to_idx.items():
    protein_to_idx2[p] = i + 1
n_proteins2 = len(protein_to_idx2)

print("n_proteins2:", n_proteins2)

fallback_prot_idx = 0


n_proteins2: 4


In [75]:
import math
import pandas as pd
import numpy as np
import duckdb
from torch_geometric.loader import DataLoader

def build_test_graphs_chunk(df_chunk):
    graphs = []
    ids = []
    bad_ids = []

    for row in df_chunk.itertuples(index=False):
        g = smiles_to_pyg(row.molecule_smiles)
        if g is None:
            bad_ids.append(int(row.id))
            continue

        prot = str(row.protein_name)
        prot_idx = protein_to_idx.get(prot, fallback_prot_idx)

        g.prot = torch.tensor(prot_idx, dtype=torch.long)
        g.id = int(row.id)
        graphs.append(g)
        ids.append(int(row.id))

    return graphs, ids, bad_ids

def predict_chunk_ensemble(models, graphs, bs=256):
    loader = DataLoader(graphs, batch_size=bs, shuffle=False)
    preds = None

    for m in models:
        p = predict_proba(m, loader)
        preds = p if preds is None else preds + p

    preds /= len(models)
    return preds


In [76]:
test_con = duckdb.connect()
total_rows = test_con.execute(f"SELECT COUNT(*) FROM '{test_path}'").fetchone()[0]
print("test rows:", total_rows)

CHUNK = 50000
n_chunks = math.ceil(total_rows / CHUNK)

out_path = "submission_gnn_kfold.csv"
import os
if os.path.exists(out_path):
    os.remove(out_path)

all_bad = 0

for k in range(n_chunks):
    off = k * CHUNK
    dfc = test_con.query(
        f"SELECT id, molecule_smiles, protein_name FROM '{test_path}' LIMIT {CHUNK} OFFSET {off}"
    ).df()

    graphs, ids_ok, bad_ids = build_test_graphs_chunk(dfc)
    all_bad += len(bad_ids)

    if len(graphs) > 0:
        p = predict_chunk_ensemble(fold_models, graphs, bs=256).astype(np.float32)
        out_ok = pd.DataFrame({"id": ids_ok, "binds": p})
    else:
        out_ok = pd.DataFrame({"id": [], "binds": []})

    out_bad = pd.DataFrame({"id": bad_ids, "binds": np.zeros(len(bad_ids), dtype=np.float32)})

    out = pd.concat([out_ok, out_bad], ignore_index=True)
    out.to_csv(out_path, index=False, mode="a", header=not os.path.exists(out_path))

    print(f"chunk {k+1}/{n_chunks} | ok={len(ids_ok)} bad={len(bad_ids)}")

test_con.close()
print("Saved:", out_path, "| bad_smiles total:", all_bad)


test rows: 1674896
chunk 1/34 | ok=50000 bad=0
chunk 2/34 | ok=50000 bad=0
chunk 3/34 | ok=50000 bad=0
chunk 4/34 | ok=50000 bad=0
chunk 5/34 | ok=50000 bad=0
chunk 6/34 | ok=50000 bad=0
chunk 7/34 | ok=50000 bad=0
chunk 8/34 | ok=50000 bad=0
chunk 9/34 | ok=50000 bad=0
chunk 10/34 | ok=50000 bad=0
chunk 11/34 | ok=50000 bad=0
chunk 12/34 | ok=50000 bad=0
chunk 13/34 | ok=50000 bad=0
chunk 14/34 | ok=50000 bad=0
chunk 15/34 | ok=50000 bad=0
chunk 16/34 | ok=50000 bad=0
chunk 17/34 | ok=50000 bad=0
chunk 18/34 | ok=50000 bad=0
chunk 19/34 | ok=50000 bad=0
chunk 20/34 | ok=50000 bad=0
chunk 21/34 | ok=50000 bad=0
chunk 22/34 | ok=50000 bad=0
chunk 23/34 | ok=50000 bad=0
chunk 24/34 | ok=50000 bad=0
chunk 25/34 | ok=50000 bad=0
chunk 26/34 | ok=50000 bad=0
chunk 27/34 | ok=50000 bad=0
chunk 28/34 | ok=50000 bad=0
chunk 29/34 | ok=50000 bad=0
chunk 30/34 | ok=50000 bad=0
chunk 31/34 | ok=50000 bad=0
chunk 32/34 | ok=50000 bad=0
chunk 33/34 | ok=50000 bad=0
chunk 34/34 | ok=24896 bad=0
Save